In [40]:
import numpy as np

In [ ]:
class enviroment():
    def __init__(self):
        """Esta clase cre el ambiente del robot movil 

        * self.numeroDeEstados -> Numero de estados
        * self.acciones -> Acciones permitidas por el robot
        * self.numeroDeAcciones -> Numero de acciones permitidas por el robot 

        """
        self.numeroDeEstados=6 #Numero de estados para el problema del robot
        self.acciones=[-1,1] #Accciones del problema del robot
        self.numeroDeAcciones=len(self.acciones) #Numero de acciones
    def reset(self):
        """Esta función resetea el robot a la posición inicial, se recomienda utilizar justo depues de inicializar el ambiente 
            ejemplo enviroment=enviroment() --> inicia el ambiente
        *self.current_state: 3, Estado actual del robot, al utiizar reset el estado del robot regresa a 3 independientemente de donde este
                                por ejemplo enviroment.reset() --> resetea al estado 3
        * terminado: False, indica que un episodio ha terminado
        * truncate: False, indica que el agente ha llegado a un punto en donde no puede avanzar y hay que terminar el estado el espisodio 
        """
        self.current_state=3
        recompensa=0
        self.terminado=False
        self.Truncado=False
        self.informacion="El robot ha empezado"
        return self.current_state, recompensa, self.terminado, self.Truncado 
    def step(self,action: int):
        """Este metodo contiene las dinamicas del ambiente

        Args:
            action (int): contiene la acción del robot, puede ser -1 o 1

        Returns:
            _int, float, bool, bool_: Estado actual, recompensa, si termino, o quedo estancado 
        """
        #RECOMPENSA------------------------------------------
        recompensa=self.reward(action)
        #----------------------------------------------------
        #CAMBIO DE ESTADO------------------------------------
        if 1<=self.current_state  and self.current_state<=4:
            self.current_state+=action
        return self.current_state, recompensa, self.terminado, self.Truncado 

    def reward(self,action):
        """Genera la recompensa del agente al tomar la acción a_i en un estado s_i

        Args:
            action (_lista_): [-1,1] 

        Returns:
            _float_: _retorna la recomepensa_
        """
        #RECOMPENSA------------------------------------------
        recompensa=0
        if self.current_state==4 and action==1:
            recompensa=5
            self.terminado=True
        if self.current_state==1 and action==-1:
            recompensa=1
            self.Truncado=True
        #----------------------------------------------------
        return recompensa

class Qiteration():
    """_Se encarga de realizar la iteración Q para el caso deterministico del robot limpiador_
    """
    def __init__(self,env,numeroDeEstados: int,numeroDeAcciones: int):
        """_Inicializa la iteración Q_

        Args:
            env (_class_): _Ambiente con las dinamicas de Transición _
            numeroDeEstados (int): _Numero de estados_
            numeroDeAcciones (int): _Numero de acciones_
        """
        self.env=env
        self.matriz_estado_accion=np.zeros((numeroDeEstados,numeroDeAcciones))
        self.gamma=0.5
        
    def Qupdate(self,estado,accion,vervose=False):
        """_Actualiza la función Q con la ecuación de Bellman_

        Args:
            estado (_int_): _estado actual del agente_
            accion (_type_): _acción tomada en el estado actual_
            vervose (bool, optional): _permite imprimit información del estado acción para saber lo que el agente esta haciendo_. Defaults to False.

        Returns:
            _type_: _La función Q descrita como matriz_
        """
        _, recompensa, terminado, Truncado =self.env.reset() #<-- Se resetea el ambiente al estado s=3
        self.env.current_state=estado #<-- Se cambia el estado al estado decidido fuera de la funcion "pueders 1,2,3..."
        current_state_1, recompensa, terminado, Truncado=self.env.step(accion)#"<---- Con el estado cambiado se realiza la acción a"
        if vervose==True:
            print("Estado antes de accion")
            print(estado)
            print("La acción tomada es !!!!!")
            print(accion)
            print("estado actual ************")
            print(current_state_1)
            print("Current Q value")
            print(self.matriz_estado_accion[current_state_1][0])
            print(self.matriz_estado_accion[current_state_1][1])
            print("Current max value")
            print(max(self.matriz_estado_accion[current_state_1][0],self.matriz_estado_accion[current_state_1][1]  ) )
            print("Recompensa!!!!!!!!!")
            print(recompensa)
        #Basicamente se esta aplicando la función de tranferencia T, en el setado "s" y la acción "a"  T(s,a)
        # A hora esto nos lleva a un nuevo estado s+1 en el cual nos hacemos la pregunta ¿Cual acción es más beneficiosa de acuerdo con mi politica?
        # Dicho de otro modo, tomamos  el producto de Gamma por el valor esperado de tomar la mejor acción en el estado s+1
        # Con esto se logea transmitir parate del valor esperado en Q(s+1,a')--> Q(s_i,a_i) lo que le da una metrica al estado s_i,a_i .   
        Q=recompensa+self.gamma*max(self.matriz_estado_accion[current_state_1][0], self.matriz_estado_accion[current_state_1][1]  )
        
        accion_to_index=int(0.5*accion+0.5)
        self.matriz_estado_accion[estado][accion_to_index]=Q #Aquí actualizamos es valor Q del estado acción  actual.
        return self.matriz_estado_accion
    

In [ ]:
"""Este codigo contiene la iteración principal 
"""
env=enviroment()
Q=Qiteration(env,env.numeroDeEstados,env.numeroDeAcciones)


for k in range(6):
    for i in range(6):
        for j in range(2):
            estado=i
            accion=env.acciones[j]
            Q_state_action_matriz=Q.Qupdate(estado,accion,vervose=False)
    print("Q-iteratio")
    print(k)
    print(Q_state_action_matriz)


Q-iteratio
0
[[0.    0.   ]
 [1.    0.   ]
 [0.5   0.   ]
 [0.25  0.   ]
 [0.125 5.   ]
 [0.    0.   ]]
Q-iteratio
1
[[0.    0.   ]
 [1.    0.25 ]
 [0.5   0.125]
 [0.25  2.5  ]
 [1.25  5.   ]
 [0.    0.   ]]
Q-iteratio
2
[[0.    0.   ]
 [1.    0.25 ]
 [0.5   1.25 ]
 [0.625 2.5  ]
 [1.25  5.   ]
 [0.    0.   ]]
Q-iteratio
3
[[0.    0.   ]
 [1.    0.625]
 [0.5   1.25 ]
 [0.625 2.5  ]
 [1.25  5.   ]
 [0.    0.   ]]
Q-iteratio
4
[[0.    0.   ]
 [1.    0.625]
 [0.5   1.25 ]
 [0.625 2.5  ]
 [1.25  5.   ]
 [0.    0.   ]]
Q-iteratio
5
[[0.    0.   ]
 [1.    0.625]
 [0.5   1.25 ]
 [0.625 2.5  ]
 [1.25  5.   ]
 [0.    0.   ]]


0
